# Phase 3: Manual Labeling

## Objectives
1. Select 50-100 representative days across different conditions
2. Manually label them: Healthy, Cloudy, Curtailment, Soiling, Investigate
3. Build gold standard validation set

In [ ]:
import sys
sys.path.insert(0, '/workspaces/O-M-Monte-Carlo')

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from src.phase3_labeling.labeler import SolarDayLabeler

sns.set_style('darkgrid')
print("Imports successful!")

## Load Feature Data

## Select Representative Days

Strategy: Use k-means clustering on features to find diverse representatives

## Interactive Labeling (Simulated)

In real usage, use `labeler.interactive_label_session()` for live prompting.
Here we'll apply labels programmatically for demonstration.